In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
import numpy as np
import matplotlib.pyplot as plt
import AutomationModule as AM
import plotly.graph_objs as go
from IPython.display import display, HTML
import plotly
from pathlib import Path

plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

In [ ]:
import h5py

def create_hdf5_from_dict(data, filename):
    with h5py.File(filename, 'w') as hdf_file:
        # Recursively traverse the dictionary and write data to the HDF5 file
        _write_dict_to_hdf5(data, hdf_file)

def _write_dict_to_hdf5(data, hdf_group):
    for key, value in data.items():
        if isinstance(value, dict):
            # If the value is a nested dictionary, create a group and call recursively
            subgroup = hdf_group.create_group(key)
            _write_dict_to_hdf5(value, subgroup)
        else:
            # If the value is not a dictionary, store it as a dataset
            hdf_group.create_dataset(key, data=value)

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:

for path_direction in [
                       "LSU Tight Beam/z_incidence"
                      
                       ]:

      folder_path = f"data/{path_direction}"
      
      for filename in os.listdir(folder_path):
              print(filename)
              if not Path(filename).suffix==".txt":
                    continue
              
              if os.path.isfile(os.path.join(folder_path, filename)):
                  file=os.path.join(folder_path, filename)
                  structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                  sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                  #Plot flux
                  #Retrieve the power flux through the monitor plane.
                  transmission0 = sim_data0['flux1'].flux
                  transmission_flux_right = sim_data['flux1'].flux/transmission0
                  transmission_flux_left = sim_data['flux2'].flux/transmission0
                  monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
                  
                  ######################################
                  #Create h5 file with data 
                  data = {
                        'transmission_right':transmission_flux_right,
                        'transmission_left':transmission_flux_left,
                        'transmission0':transmission0,
                        'lambdas':monitor_lambdas,
                        'decay':structure_1.final_decay
                  }
                  store_path = f"output/{path_direction}/Data"
                  if not os.path.exists(store_path):
                        os.makedirs(store_path)
                        print(f"Folder '{store_path}' created successfully.")
      
                  # Specify the filename for the HDF5 file
                  fileh5 = store_path+"/"+structure_1.description + '_decay_'+str(structure_1.final_decay)+'.h5'
      
                  # Create the HDF5 file from the dictionary
                  create_hdf5_from_dict(data, fileh5)
                 
